# Домашнее задание 1: Feature Engineering

Необходимо запустить практическую часть занятия- файл находиться в материалых, и посмотреть самому то, о чём говорили на лекции.
По образу практики, попробуйте создать искусственный датасет с лишними столбцами.
Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель.
Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown.
Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Попробуем прогнозировать стоимость квартиры

In [32]:
# Создаём сэмпл
n_samples = 500

#престижность района от 0 до 10, учитывающий удаленность от центра, наличие парков, ТЦ и т.д.
priority = np.random.choice(11, n_samples)
# удаленность от метро в км. (предполагаем не более 10 км)
metro = np.random.choice(11, n_samples)
# кол-во комнат (предполагаем не более 5)
rooms = np.random.choice(6, n_samples)
# квадратура
square = np.random.choice(301, n_samples)
# ср. цена за метр в данном районе
avg_price = np.random.choice(400000, n_samples)
# этаж
floor = np.random.choice(25, n_samples)

price = avg_price * square - 500000 * metro + 100000 * rooms + 50000 * floor + 300000 * priority

data = pd.DataFrame({'priority': priority, 'metro': metro, 'rooms': rooms, 'square': square, 'avg_price': avg_price, 'floor': floor, 'price':price})
data.head(5)

,priority,metro,rooms,square,avg_price,floor,price
0,10,4,2,297,308450,9,93259650
1,8,3,2,272,45444,10,13960768
2,10,3,1,245,147972,14,38553140
3,7,7,4,28,164251,3,3749028
4,7,7,4,269,48366,8,12410454


Пишем линейную модель:

In [33]:
from sklearn.metrics import mean_absolute_error

X = data[['priority', 'metro', 'rooms', 'square', 'avg_price', 'floor']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['priority', 'metro', 'rooms', 'square', 'avg_price', 'floor']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 4.49621709e+05 -3.21369198e+05 -1.05420674e+05  1.95955481e+05
  1.50237267e+02  6.83147359e+04]
Bias: -30643422.757812917
Error: 7565689.49830871


Результаты не очень, попробуем убрать часть переменных:

In [35]:
X = data[['priority', 'square', 'avg_price', 'floor']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['priority', 'square', 'avg_price', 'floor']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [4.50308929e+05 1.96272058e+05 1.50097332e+02 7.39417047e+04]
Bias: -32659793.47729347
Error: 7689868.535270695


Почти то же самое, добавим новый признак:

In [37]:
data['prestige'] = data['priority'] * data['metro']
data.head(5)

,priority,metro,rooms,square,avg_price,floor,price,prestige
0,10,4,2,297,308450,9,93259650,40
1,8,3,2,272,45444,10,13960768,24
2,10,3,1,245,147972,14,38553140,30
3,7,7,4,28,164251,3,3749028,49
4,7,7,4,269,48366,8,12410454,49


In [39]:
X = data[['prestige']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['prestige']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [26974.3161516]
Bias: 29053961.286436748
Error: 21657493.05168028


Результаты сильно хуже - недостаточно одного такого признака